In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Meetup - 28.02.23/data.csv')
df

,name,text
0,tamir,I have two brothers and a beautiful partner.\n...
1,adi,I have two parents and I live in Rishon Lezion...
2,ishai,I have two children and I am married. \nI live...
3,simona,I have a debt that I can't repay and I don't k...


In [ ]:
df['text'] = (df['name']+ ' - ' + df['text'])
df = df.iloc[:,0:]

In [ ]:
#df = df.explode('text')
#df = df[df['text'].str.split().map(len) > 10]
df

,name,text
0,tamir,tamir - I have two brothers and a beautiful pa...
1,adi,adi - I have two parents and I live in Rishon ...
2,ishai,ishai - I have two children and I am married. ...
3,simona,simona - I have a debt that I can't repay and ...


In [ ]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=9381508138725dbc22dffde1af5d01824b604f03cc5a863524da72d2104a49f8
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c303e8ff175416d5c1c9c03a2e83ebf7525a99
Successfully built openai


In [ ]:
import openai
OPENAI_API_KEY = 'your key'

openai.api_key = OPENAI_API_KEY 

In [ ]:
def get_embedding(texts, model="text-embedding-ada-002"):
   texts = [text.replace('\n', ' ') for text in texts]
   return [res['embedding'] for res in openai.Embedding.create(input = texts, model=model)['data']]

In [ ]:
#embs = get_embedding(['Gampling','hi'])

In [ ]:
#len(embs)

In [ ]:
#from tqdm import trange
batch_size = 100
embeddings = []

for x in range(0, len(df), batch_size):
  embeddings += get_embedding(df.iloc[x:x+batch_size]['text'])

In [ ]:
df['embedding'] = embeddings

<ipython-input-17-95388b743f6f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embedding'] = embeddings


In [ ]:
df

,name,text,embedding
0,tamir,tamir - I have two brothers and a beautiful pa...,"[-0.012466193176805973, -0.01888096332550049, ..."
1,adi,adi - I have two parents and I live in Rishon ...,"[0.0037490129470825195, 0.010697444900870323, ..."
2,ishai,ishai - I have two children and I am married. ...,"[-0.029545078054070473, -0.0030164928175508976..."
3,simona,simona - I have a debt that I can't repay and ...,"[-0.0065046935342252254, -0.012495001778006554..."


In [ ]:
query = 'who doesnt work ?'  
query_embedding = get_embedding([query])[0]

In [ ]:
best_idx = df['embedding'].map(
    lambda emb: np.dot(emb, query_embedding) / (
        np.linalg.norm(emb) * np.linalg.norm(query_embedding)
    )
).argmax()

best_answer = df.iloc[best_idx]['text']

In [ ]:
prompt = (
    "Here's a piece of text:\n" +
    best_answer + '\n\n' +
    'I have a question about this text: ' + query +
    'Please answer in a concise manner'
)

print(prompt)

Here's a piece of text:
ishai - I have two children and I am married. 
I live in Petah Tikva and love to learn new things. 
I recently got fired from my job and am currently looking for a job.

I have a question about this text: who doesnt work ?Please answer in a concise manner
